In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense

In [4]:
class AlexNet():
    
    def __init__(self):
        self.model = Sequential()
        self.model.add(Conv2D(96, input_shape=(224,224,3), kernel_size=11, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(256, kernel_size=5, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(384, kernel_size=3, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Conv2D(256, kernel_size=3, activation="relu"))
        self.model.add(MaxPool2D((3,3)))
        self.model.add(Dense(4096, activation="leaky_relu"))
        self.model.add(Dense(4096, activation="leaky_relu"))
        self.model.add(Dense(17, activation="linear"))
        self.model.compile(optimizer="sgd", loss="root_mean_squared_error", metrics="accuracy")

In [7]:
model = AlexNet()
#model.model.summary()

In [8]:
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

In [29]:
import pathlib

data_dir = pathlib.Path('images')
dataset = tf.data.Dataset.list_files("images/*.jpg")

In [31]:
print(len(dataset))

24984


In [36]:
import json

with open('train.json') as file:
    data = json.load(file)
y_train = data

In [37]:
len(y_train)

22246